## Test NIRSpec IFS data in cubeviz

In [ ]:
#to use 100% of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import time

import specutils
from specutils import Spectrum1D

import jdaviz
from jdaviz import CubeViz

import astropy
from astropy.io import fits
from astropy import units as u
from astropy.nddata import StdDevUncertainty
import astropy.wcs

import numpy as np

Check versions. Works with:
- jdaviz: 2.0.1.dev106+g392e1b6
- astropy: 4.3.post1
- specutils: 1.4.2.dev42+g08b2a7f3

In [ ]:
print("jdaviz:",jdaviz.__version__)
print("astropy:",astropy.__version__)
print("specutils:",specutils.__version__)

## Load a pipeline-processed JWST simulated cube as example

In [ ]:
from astropy.utils.data import download_file
filename = "https://stsci.box.com/shared/static/ff0bj31acot1272x5qq2clbmto3s6v4f.fits"
ifucube_file = download_file(filename, cache=True)
#cubeviz.load_data(fn)

In [ ]:
cubeviz2 = CubeViz()
cubeviz2.app

In [ ]:
cubeviz2.app.load_data(ifucube_file)

## Explore the IPS file as is

In [ ]:
filename = "./3004_IPS_18660_badrc_CUBE_R3D.fits"

In [ ]:
hdulist = fits.open(filename)
hdulist.info()

In [ ]:
fnhead_sci = hdulist['DATA'].header
fnhead_err = hdulist['ERR'].header
fnhead_dq = hdulist['QUALITY'].header
hdulist['DATA'].header

## Here I create Spectrum1D object

In [ ]:
#create Spectrum1D object from data in cube
wave = np.arange(fnhead_sci['CRVAL3'], fnhead_sci['CRVAL3'] + float(fnhead_sci['NAXIS3'])*fnhead_sci['CDELT3'], fnhead_sci['CDELT3']) * 1.E6 * u.um
print(wave)
print(len(wave))
print(fnhead_sci['NAXIS3'])

In [ ]:
#create a WCS object instead
wcs_dict = {
'CTYPE1': 'WAVE    ', 'CUNIT1': 'um', 'CDELT1': fnhead_sci['CDELT3'] * 1E6, 'CRPIX1': fnhead_sci['CRPIX3'], 'CRVAL1': fnhead_sci['CRVAL3'] * 1E6, 'NAXIS1': fnhead_sci['NAXIS3'],
'CTYPE2': 'HPLT-TAN', 'CUNIT2': 'deg', 'CDELT2': fnhead_sci['CDELT1'], 'CRPIX2': fnhead_sci['CRPIX1'], 'CRVAL2': fnhead_sci['CRVAL1'], 'NAXIS2': fnhead_sci['NAXIS1'],
'CTYPE3': 'HPLN-TAN', 'CUNIT3': 'deg', 'CDELT3': fnhead_sci['CDELT2'], 'CRPIX3': fnhead_sci['CRPIX2'], 'CRVAL3': fnhead_sci['CRVAL2'], 'NAXIS3': fnhead_sci['NAXIS2']}
input_wcs = astropy.wcs.WCS(wcs_dict)

In [ ]:
#rearrange the data
data = hdulist['DATA'].data
print(data.shape)

new_data = np.zeros((35,30,3817))
for i in range (0,35):
    for j in range (0,30):
        for k in range (0,3817):
            new_data[i,j,k] = data[k,i,j]

print(new_data.shape)


In [ ]:
#full cube with wcs
cube = Spectrum1D(flux=new_data * u.uJy,
                  wcs=input_wcs,
                  uncertainty = StdDevUncertainty(new_data*0.01*u.uJy))

### Open a new cubeviz instance to try these out

In [ ]:
cubeviz3 = CubeViz()
cubeviz3.app

In [ ]:
#this opens the full cube in the form of a Spectrum1D object
cubeviz3.app.load_data(cube, data_label="cube")